# Analyzing your model with TensorFlow Model Analysis and the What-If Tool

## TFMA

In [1]:
import tensorflow_model_analysis as tfma
import tensorflow as tf

import sys
import os

# stop tf warnings going everywhere
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:
# add project to the python path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
_EVAL_DATA_FILE = 'data_tfrecord-00000-of-00001'
_MODEL_DIR = 'serving_model_dir_2000_steps/'

Convert the model into the required format for TFMA

In [6]:
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR, tags=[tf.saved_model.SERVING])

In [4]:
slices = [tfma.slicer.SingleSliceSpec(),
          tfma.slicer.SingleSliceSpec(columns=['product'])]

In [5]:
eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives')
              ])])

In [ ]:
eval_config_plot=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
    metrics_specs=tfma.metrics.specs_from_metrics([
        tfma.metrics.ConfusionMatrixPlot(),
    ]))

In [ ]:
# this takes a minute
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config_plot,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_plot",
    file_format='tfrecords',
    slice_spec = slices)

In [ ]:
tfma.view.render_slicing_metrics(eval_result)

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_spec=slices[1])

In [ ]:
#
tfma.view.render_plot(eval_result)

## Adding a custom metric

In [ ]:
# custom Keras metric
# https://github.com/tensorflow/model-analysis/blob/master/g3doc/metrics.md
class MyMetric(tf.keras.metrics.Mean):

  def __init__(self, name='my_metric', dtype=None):
    super(MyMetric, self).__init__(name=name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    return super(MyMetric, self).update_state(
        y_pred, sample_weight=sample_weight)

In [ ]:
# https://github.com/tensorflow/model-analysis/blob/master/g3doc/post_export_metrics.md

## Compare 2 models

In [ ]:
eval_shared_model_2 = tfma.default_eval_shared_model(
    eval_saved_model_path='serving_model_dir_150_steps/', tags=[tf.saved_model.SERVING])

eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model_2,
    eval_config=eval_config,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_150_steps",
    file_format='tfrecords',
    slice_spec = slices)

In [ ]:
eval_results_from_disk = tfma.load_eval_results(
    ['./eval_result_2000_steps','./eval_result_150_steps'], tfma.constants.MODEL_CENTRIC_MODE)

In [ ]:
#tfma.view.render_time_series(eval_results_from_disk, slices[0])
# many js errors

## Fairness indicators

In [ ]:
# https://github.com/tensorflow/tensorboard/blob/master/docs/fairness-indicators.md
!pip install tensorboard_plugin_fairness_indicators

In [17]:
metrics_callbacks = \
    [tfma.post_export_metrics.fairness_indicators(thresholds=[0.25, 0.5, 0.75])]

In [18]:
eval_shared_model_fairness = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR,
    add_metrics_callbacks=metrics_callbacks,
     tags=[tf.saved_model.SERVING])

In [19]:
eval_config_fairness=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives'),
                  tfma.MetricConfig(class_name='FairnessIndicators', config='{"thresholds":[0.25, 0.5, 0.75]}')
              ])])

In [20]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model_fairness,
    eval_config=eval_config_fairness,
    data_location=_EVAL_DATA_FILE,
    output_path="./eval_result_fairness",
    file_format='tfrecords',
    slice_spec = slices)

In [21]:
from tensorboard_plugin_fairness_indicators import summary_v2

In [22]:
writer = tf.summary.create_file_writer('./fairness_indicator_logs')
with writer.as_default():
    summary_v2.FairnessIndicators('./eval_result_fairness', step=1)
writer.close()

In [23]:
%load_ext tensorboard
%tensorboard --logdir=./fairness_indicator_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 58921), started 2:04:32 ago. (Use '!kill 58921' to kill it.)

## The What-If Tool

In [ ]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [ ]:
eval_data = tf.data.TFRecordDataset(_EVAL_DATA_FILE)

In [ ]:
eval_examples = [tf.train.Example.FromString(d.numpy()) for d in eval_data.take(1000)]

In [ ]:
model = tf.saved_model.load(export_dir=_MODEL_DIR)

In [ ]:
def predict(examples):
    preds = model.signatures['serving_default'](examples=tf.constant([example.SerializeToString() for example in examples]))
    return preds['outputs'].numpy()

In [ ]:
config_builder = WitConfigBuilder(eval_examples).set_custom_predict_fn(predict)

In [ ]:
WitWidget(config_builder)

### Debugging

In [ ]:
# works with 2.1
!pip show tensorflow

In [ ]:
# works with >0.21.3
!pip show tensorflow_model_analysis

In [ ]:
# works with 1.6.0
!pip show witwidget

In [ ]:
# may need to run this every time
!jupyter nbextension install --py --symlink --sys-prefix witwidget

!jupyter nbextension enable witwidget --py --sys-prefix 

# then refresh browser page

In [ ]:
# may need to run this every time

!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# then refresh browser page

In [ ]:
!pip install widgetsnbextension

In [ ]:
!pip install -U ipywidgets

In [ ]:
!jupyter nbextension list

In [ ]:
!jupyter serverextension list